In [10]:
import os
import pandas as pd
path = r"C:\Users\Wong\Desktop\Dissertation\Imputed_dataset"

Haltwhistle_rainfall = pd.read_csv(os.path.join(path, 'Haltwhistle_rainfall.csv'))
Haltwhistle_water = pd.read_csv(os.path.join(path, 'Haltwhistle_water.csv'))

In [11]:
Haltwhistle_rainfall['dateTime'] = pd.to_datetime(Haltwhistle_rainfall['dateTime'])
Haltwhistle_water['dateTime'] = pd.to_datetime(Haltwhistle_water['dateTime'])

In [12]:
Haltwhistle_water.set_index('dateTime', inplace=True)
Haltwhistle_rainfall.set_index('dateTime', inplace=True)

In [13]:
start_date = pd.to_datetime('13/11/2020 00:00:00', format='%d/%m/%Y %H:%M:%S')
end_date = pd.to_datetime('3/2/2024 00:00:00', format='%d/%m/%Y %H:%M:%S')
Haltwhistle_water_subset = Haltwhistle_water.loc[(Haltwhistle_water.index >= start_date) & (Haltwhistle_water.index <= end_date)]
Haltwhistle_rainfall_subset = Haltwhistle_rainfall.loc[(Haltwhistle_rainfall.index >= start_date) & (Haltwhistle_rainfall.index <= end_date)]

In [14]:
Haltwhistle_water_subset.shape

(112993, 1)

In [15]:
Haltwhistle_rainfall_subset

,value
dateTime,
2020-11-13 00:00:00,0.0
2020-11-13 00:15:00,0.0
2020-11-13 00:30:00,0.0
2020-11-13 00:45:00,0.0
2020-11-13 01:00:00,0.0
...,...
2024-02-02 23:00:00,0.0
2024-02-02 23:15:00,0.0
2024-02-02 23:30:00,0.0


In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error
import tensorflow as tf
from keras import Model
from keras.layers import Input, Dense, Dropout
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error, mean_absolute_error
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.layers import Dense, Dropout

In [17]:
merged_data = pd.merge(Haltwhistle_rainfall_subset, Haltwhistle_water_subset, on='dateTime', how='inner')

In [18]:
merged_data.shape

(112993, 2)

In [19]:
train_merged, valid_merged, test_merged = merged_data[1:90394], merged_data[90394:101693],merged_data[101693:112993]

In [20]:
train_Haltwhistle = train_merged
scalers_Haltwhistle={}
for i in train_merged.columns:
    scaler = MinMaxScaler(feature_range=(-1,1))
    transform = scaler.fit_transform(train_Haltwhistle[i].values.reshape(-1,1))
    transform=np.reshape(transform,len(transform))
    scalers_Haltwhistle['scaler_'+ i] = scaler
    train_Haltwhistle[i]=transform
test_Haltwhistle = test_merged
for i in test_merged.columns:
    scaler = scalers_Haltwhistle['scaler_'+i]
    transform = scaler.transform(test_Haltwhistle[i].values.reshape(-1,1))
    transform=np.reshape(transform,len(transform))
    scalers_Haltwhistle['scaler_'+i] = scaler
    test_Haltwhistle[i]=transform
valid_Haltwhistle = valid_merged
for i in valid_merged.columns:
    scaler = scalers_Haltwhistle['scaler_'+i]
    transform = scaler.transform(valid_Haltwhistle[i].values.reshape(-1,1))
    transform=np.reshape(transform,len(transform))
    scalers_Haltwhistle['scaler_'+i] = scaler
    valid_Haltwhistle[i]=transform

C:\Users\Wong\AppData\Local\Temp\ipykernel_40492\636357226.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_Haltwhistle[i]=transform
C:\Users\Wong\AppData\Local\Temp\ipykernel_40492\636357226.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_Haltwhistle[i]=transform
C:\Users\Wong\AppData\Local\Temp\ipykernel_40492\636357226.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cavea

In [21]:
def split(series, n_past, n_future):
    X, y = list(), list()
    for start in range(len(series)):
        p_end = start + n_past
        f_end = p_end + n_future
        if f_end > len(series):
            break
        past_window, future_window = series[start:p_end, :], series[p_end:f_end, :]
        X.append(past_window)
        y.append(future_window)
    return np.array(X), np.array(y)

In [22]:
n_past = 96
n_future = 48 
n_features = 2
X_train, y_train = split(train_Haltwhistle.values,n_past, n_future)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1],n_features))
y_train = y_train.reshape((y_train.shape[0], y_train.shape[1], n_features))
X_test, y_test = split(test_Haltwhistle.values,n_past, n_future)
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1],n_features))
y_test = y_test.reshape((y_test.shape[0], y_test.shape[1], n_features))
X_valid, y_valid = split(valid_Haltwhistle.values,n_past, n_future)
X_valid = X_valid.reshape((X_valid.shape[0], X_valid.shape[1],n_features))
y_valid = y_valid.reshape((y_valid.shape[0], y_valid.shape[1], n_features))

In [23]:
e_in = tf.keras.layers.Input(shape=(n_past, n_features))
e_l1 = tf.keras.layers.LSTM(100,return_sequences = True, return_state=True)
e_outs1 = e_l1(e_in)
e_states1 = e_outs1[1:]
e_l2 = tf.keras.layers.LSTM(100, return_state=True)
e_outs2 = e_l2(e_outs1[0])
e_states2 = e_outs2[1:]

d_in = tf.keras.layers.RepeatVector(n_future)(e_outs2[0])

d_l1 = tf.keras.layers.LSTM(100, return_sequences=True)(d_in,initial_state = e_states1)
d_l2 = tf.keras.layers.LSTM(100, return_sequences=True)(d_l1,initial_state = e_states2)
d_outs2 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(n_features))(d_l2)

model_e2d2_batch_32_96_in_Halt= tf.keras.models.Model(e_in,d_outs2)

In [24]:
reduce_learning_rate = tf.keras.callbacks.LearningRateScheduler(lambda x: 1e-3 * 0.90 ** x)
model_e2d2_batch_32_96_in_Halt.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.Huber())
history_e2d2_batch_32_96_in_Halt=model_e2d2_batch_32_96_in_Halt.fit(X_train,y_train,epochs=25,validation_data=(X_valid,y_valid),batch_size=32,verbose=1,callbacks=[reduce_learning_rate])

Epoch 1/25
2821/2821 [==============================] - 729s 256ms/step - loss: 0.0029 - val_loss: 0.0042 - lr: 0.0010
Epoch 2/25
2821/2821 [==============================] - 558s 198ms/step - loss: 0.0019 - val_loss: 0.0026 - lr: 9.0000e-04
Epoch 3/25
2821/2821 [==============================] - 567s 201ms/step - loss: 0.0018 - val_loss: 0.0026 - lr: 8.1000e-04
Epoch 4/25
2821/2821 [==============================] - 587s 208ms/step - loss: 0.0016 - val_loss: 0.0024 - lr: 7.2900e-04
Epoch 5/25
2821/2821 [==============================] - 523s 186ms/step - loss: 0.0016 - val_loss: 0.0025 - lr: 6.5610e-04
Epoch 6/25
2821/2821 [==============================] - 518s 184ms/step - loss: 0.0015 - val_loss: 0.0025 - lr: 5.9049e-04
Epoch 7/25
2821/2821 [==============================] - 534s 189ms/step - loss: 0.0015 - val_loss: 0.0023 - lr: 5.3144e-04
Epoch 8/25
2821/2821 [==============================] - 595s 211ms/step - loss: 0.0015 - val_loss: 0.0024 - lr: 4.7830e-04
Epoch 9/25
2821/2821

In [25]:
import pickle
with open('history_e2d2_batch_32_96_in_Halt.pkl', 'wb') as file:
    pickle.dump(history_e2d2_batch_32_96_in_Halt.history, file)

In [26]:
from keras.models import load_model
model_e2d2_batch_32_96_in_Halt.save('model_e2d2_batch_32_96_in_Halt.h5')

In [27]:
pred_e2d2_batch_32_96_in_Halt=model_e2d2_batch_32_96_in_Halt.predict(X_test)

349/349 [==============================] - 27s 71ms/step


In [28]:
for index,i in enumerate(train_merged.columns):
    scaler = scalers_Haltwhistle['scaler_'+i]
    pred_e2d2_batch_32_96_in_Halt[:,:,index]=scaler.inverse_transform(pred_e2d2_batch_32_96_in_Halt[:,:,index])
    y_train[:,:,index]=scaler.inverse_transform(y_train[:,:,index])
    y_test[:,:,index]=scaler.inverse_transform(y_test[:,:,index])
    y_valid[:,:,index]=scaler.inverse_transform(y_valid[:,:,index])

In [29]:
def calculate_nse(y_true, y_pred):
    
    return 1 - np.sum((y_true - y_pred) ** 2) / np.sum((y_true - np.mean(y_true)) ** 2)

for index, i in enumerate(train_merged.columns):
    print(i)
    for j in range(1, 49):
        print("time ", j, ":")
        mae = mean_absolute_error(y_test[:, j - 1, index], pred_e2d2_batch_32_96_in_Halt[:, j - 1, index])
        mse = mean_squared_error(y_test[:, j - 1, index], pred_e2d2_batch_32_96_in_Halt[:, j - 1, index])
        nse = calculate_nse(y_test[:, j - 1, index], pred_e2d2_batch_32_96_in_Halt[:, j - 1, index])
        print("MAE: ", mae)
        print("MSE: ", mse)
        print("NSE: ", nse)
    print()

value_x
time  1 :
MAE:  0.04919486925402582
MSE:  0.01194103225921912
NSE:  0.2919079973021427
time  2 :
MAE:  0.05181723312845664
MSE:  0.013159795639916176
NSE:  0.21963647300521194
time  3 :
MAE:  0.05009833743433976
MSE:  0.014226621703287874
NSE:  0.15625931593917275
time  4 :
MAE:  0.05339711683672838
MSE:  0.015040098450702621
NSE:  0.10801431149284868
time  5 :
MAE:  0.05480514602271355
MSE:  0.015708230398788027
NSE:  0.06838929589338327
time  6 :
MAE:  0.057054974403049706
MSE:  0.016255545904225788
NSE:  0.03592956170023831
time  7 :
MAE:  0.058074793170683814
MSE:  0.016678154637424305
NSE:  0.010865833355188426
time  8 :
MAE:  0.05846583286985292
MSE:  0.01702000016037692
NSE:  -0.00940805748092699
time  9 :
MAE:  0.05854693070699145
MSE:  0.017280754667151052
NSE:  -0.024872669565657146
time  10 :
MAE:  0.0585160620079851
MSE:  0.01754887738302114
NSE:  -0.04077426928614414
time  11 :
MAE:  0.058422380127530274
MSE:  0.01771956700387608
NSE:  -0.051041166811540695
time  1

In [30]:
n_past = 96
n_future = 48 
n_features = 2
X_train, y_train = split(train_Haltwhistle.values,n_past, n_future)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1],n_features))
y_train = y_train.reshape((y_train.shape[0], y_train.shape[1], n_features))
X_test, y_test = split(test_Haltwhistle.values,n_past, n_future)
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1],n_features))
y_test = y_test.reshape((y_test.shape[0], y_test.shape[1], n_features))
X_valid, y_valid = split(valid_Haltwhistle.values,n_past, n_future)
X_valid = X_valid.reshape((X_valid.shape[0], X_valid.shape[1],n_features))
y_valid = y_valid.reshape((y_valid.shape[0], y_valid.shape[1], n_features))

In [31]:
e_in = tf.keras.layers.Input(shape=(n_past, n_features))
e_l1 = tf.keras.layers.LSTM(100,return_sequences = True, return_state=True)
e_outs1 = e_l1(e_in)
e_states1 = e_outs1[1:]
e_l2 = tf.keras.layers.LSTM(100, return_state=True)
e_outs2 = e_l2(e_outs1[0])
e_states2 = e_outs2[1:]

d_in = tf.keras.layers.RepeatVector(n_future)(e_outs2[0])

d_l1 = tf.keras.layers.LSTM(100, return_sequences=True)(d_in,initial_state = e_states1)
d_l2 = tf.keras.layers.LSTM(100, return_sequences=True)(d_l1,initial_state = e_states2)
d_outs2 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(n_features))(d_l2)

model_e2d2_batch_64_96_in_Halt= tf.keras.models.Model(e_in,d_outs2)

In [32]:
reduce_learning_rate = tf.keras.callbacks.LearningRateScheduler(lambda x: 1e-3 * 0.90 ** x)
model_e2d2_batch_64_96_in_Halt.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.Huber())
history_e2d2_batch_64_96_in_Halt=model_e2d2_batch_64_96_in_Halt.fit(X_train,y_train,epochs=25,validation_data=(X_valid,y_valid),batch_size=64,verbose=1,callbacks=[reduce_learning_rate])

Epoch 1/25
1411/1411 [==============================] - 542s 377ms/step - loss: 0.0034 - val_loss: 0.0031 - lr: 0.0010
Epoch 2/25
1411/1411 [==============================] - 601s 426ms/step - loss: 0.0020 - val_loss: 0.0036 - lr: 9.0000e-04
Epoch 3/25
1411/1411 [==============================] - 527s 374ms/step - loss: 0.0018 - val_loss: 0.0030 - lr: 8.1000e-04
Epoch 4/25
1411/1411 [==============================] - 511s 362ms/step - loss: 0.0017 - val_loss: 0.0025 - lr: 7.2900e-04
Epoch 5/25
1411/1411 [==============================] - 513s 363ms/step - loss: 0.0016 - val_loss: 0.0025 - lr: 6.5610e-04
Epoch 6/25
1411/1411 [==============================] - 544s 386ms/step - loss: 0.0016 - val_loss: 0.0025 - lr: 5.9049e-04
Epoch 7/25
1411/1411 [==============================] - 522s 370ms/step - loss: 0.0015 - val_loss: 0.0025 - lr: 5.3144e-04
Epoch 8/25
1411/1411 [==============================] - 463s 328ms/step - loss: 0.0015 - val_loss: 0.0025 - lr: 4.7830e-04
Epoch 9/25
1411/1411

In [33]:
with open('history_e2d2_batch_64_96_in_Halt.pkl', 'wb') as file:
    pickle.dump(history_e2d2_batch_64_96_in_Halt.history, file)

In [34]:
from keras.models import load_model
model_e2d2_batch_64_96_in_Halt.save('model_e2d2_batch_64_96_in_Halt.h5')

In [35]:
pred_e2d2_batch_64_96_in_Halt=model_e2d2_batch_64_96_in_Halt.predict(X_test)

349/349 [==============================] - 23s 61ms/step


In [36]:
for index,i in enumerate(train_merged.columns):
    scaler = scalers_Haltwhistle['scaler_'+i]
    pred_e2d2_batch_64_96_in_Halt[:,:,index]=scaler.inverse_transform(pred_e2d2_batch_64_96_in_Halt[:,:,index])
    y_train[:,:,index]=scaler.inverse_transform(y_train[:,:,index])
    y_test[:,:,index]=scaler.inverse_transform(y_test[:,:,index])
    y_valid[:,:,index]=scaler.inverse_transform(y_valid[:,:,index])

In [37]:
def calculate_nse(y_true, y_pred):
    
    return 1 - np.sum((y_true - y_pred) ** 2) / np.sum((y_true - np.mean(y_true)) ** 2)

for index, i in enumerate(train_merged.columns):
    print(i)
    for j in range(1, 49):
        print("time ", j, ":")
        mae = mean_absolute_error(y_test[:, j - 1, index], pred_e2d2_batch_64_96_in_Halt[:, j - 1, index])
        mse = mean_squared_error(y_test[:, j - 1, index], pred_e2d2_batch_64_96_in_Halt[:, j - 1, index])
        nse = calculate_nse(y_test[:, j - 1, index], pred_e2d2_batch_64_96_in_Halt[:, j - 1, index])
        print("MAE: ", mae)
        print("MSE: ", mse)
        print("NSE: ", nse)
    print()

value_x
time  1 :
MAE:  0.053812019656377
MSE:  0.011924128103119802
NSE:  0.2929103978891643
time  2 :
MAE:  0.05192838183993923
MSE:  0.012923038697395731
NSE:  0.23367593742860737
time  3 :
MAE:  0.05103303360193234
MSE:  0.013930997325831742
NSE:  0.17379196140216757
time  4 :
MAE:  0.05527942597123472
MSE:  0.014675801338064396
NSE:  0.12961974259443787
time  5 :
MAE:  0.056797590368567824
MSE:  0.015207586299960031
NSE:  0.09808108100063251
time  6 :
MAE:  0.05790350383954461
MSE:  0.015616831150436305
NSE:  0.07380992673149056
time  7 :
MAE:  0.05880380053775354
MSE:  0.015885246498677504
NSE:  0.0578909718129722
time  8 :
MAE:  0.059359001820581285
MSE:  0.016062617935107066
NSE:  0.04737157372766698
time  9 :
MAE:  0.05978184978071054
MSE:  0.016235545980690543
NSE:  0.037115700582450395
time  10 :
MAE:  0.06016582655011299
MSE:  0.016424397786806894
NSE:  0.025915434250640246
time  11 :
MAE:  0.06052552911557209
MSE:  0.0165409692615511
NSE:  0.01886769416817291
time  12 :
MA

In [38]:
n_past = 96
n_future = 48 
n_features = 2
X_train, y_train = split(train_Haltwhistle.values,n_past, n_future)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1],n_features))
y_train = y_train.reshape((y_train.shape[0], y_train.shape[1], n_features))
X_test, y_test = split(test_Haltwhistle.values,n_past, n_future)
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1],n_features))
y_test = y_test.reshape((y_test.shape[0], y_test.shape[1], n_features))
X_valid, y_valid = split(valid_Haltwhistle.values,n_past, n_future)
X_valid = X_valid.reshape((X_valid.shape[0], X_valid.shape[1],n_features))
y_valid = y_valid.reshape((y_valid.shape[0], y_valid.shape[1], n_features))

In [39]:
e_in = tf.keras.layers.Input(shape=(n_past, n_features))
e_l1 = tf.keras.layers.LSTM(100,return_sequences = True, return_state=True)
e_outs1 = e_l1(e_in)
e_states1 = e_outs1[1:]
e_l2 = tf.keras.layers.LSTM(100, return_state=True)
e_outs2 = e_l2(e_outs1[0])
e_states2 = e_outs2[1:]

d_in = tf.keras.layers.RepeatVector(n_future)(e_outs2[0])

d_l1 = tf.keras.layers.LSTM(100, return_sequences=True)(d_in,initial_state = e_states1)
d_l2 = tf.keras.layers.LSTM(100, return_sequences=True)(d_l1,initial_state = e_states2)
d_outs2 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(n_features))(d_l2)

model_e2d2_batch_128_96_in_Halt= tf.keras.models.Model(e_in,d_outs2)

In [40]:
reduce_learning_rate = tf.keras.callbacks.LearningRateScheduler(lambda x: 1e-3 * 0.90 ** x)
model_e2d2_batch_128_96_in_Halt.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.Huber())
history_e2d2_batch_128_96_in_Halt=model_e2d2_batch_128_96_in_Halt.fit(X_train,y_train,epochs=25,validation_data=(X_valid,y_valid),batch_size=128,verbose=1,callbacks=[reduce_learning_rate])

Epoch 1/25
706/706 [==============================] - 388s 540ms/step - loss: 0.0046 - val_loss: 0.0038 - lr: 0.0010
Epoch 2/25
706/706 [==============================] - 370s 524ms/step - loss: 0.0021 - val_loss: 0.0029 - lr: 9.0000e-04
Epoch 3/25
706/706 [==============================] - 366s 519ms/step - loss: 0.0019 - val_loss: 0.0030 - lr: 8.1000e-04
Epoch 4/25
706/706 [==============================] - 378s 536ms/step - loss: 0.0018 - val_loss: 0.0032 - lr: 7.2900e-04
Epoch 5/25
706/706 [==============================] - 371s 526ms/step - loss: 0.0017 - val_loss: 0.0037 - lr: 6.5610e-04
Epoch 6/25
706/706 [==============================] - 375s 532ms/step - loss: 0.0017 - val_loss: 0.0028 - lr: 5.9049e-04
Epoch 7/25
706/706 [==============================] - 373s 529ms/step - loss: 0.0016 - val_loss: 0.0025 - lr: 5.3144e-04
Epoch 8/25
706/706 [==============================] - 376s 533ms/step - loss: 0.0016 - val_loss: 0.0028 - lr: 4.7830e-04
Epoch 9/25
706/706 [================

In [41]:
with open('history_e2d2_batch_128_96_in_Halt.pkl', 'wb') as file:
    pickle.dump(history_e2d2_batch_128_96_in_Halt.history, file)

In [42]:
from keras.models import load_model
model_e2d2_batch_128_96_in_Halt.save('model_e2d2_batch_128_96_in_Halt.h5')

In [43]:
pred_e2d2_batch_128_96_in_Halt=model_e2d2_batch_128_96_in_Halt.predict(X_test)

349/349 [==============================] - 25s 69ms/step


In [44]:
for index,i in enumerate(train_merged.columns):
    scaler = scalers_Haltwhistle['scaler_'+i]
    pred_e2d2_batch_128_96_in_Halt[:,:,index]=scaler.inverse_transform(pred_e2d2_batch_128_96_in_Halt[:,:,index])
    y_train[:,:,index]=scaler.inverse_transform(y_train[:,:,index])
    y_test[:,:,index]=scaler.inverse_transform(y_test[:,:,index])
    y_valid[:,:,index]=scaler.inverse_transform(y_valid[:,:,index])

In [45]:
def calculate_nse(y_true, y_pred):
    
    return 1 - np.sum((y_true - y_pred) ** 2) / np.sum((y_true - np.mean(y_true)) ** 2)

for index, i in enumerate(train_merged.columns):
    print(i)
    for j in range(1, 49):
        print("time ", j, ":")
        mae = mean_absolute_error(y_test[:, j - 1, index], pred_e2d2_batch_128_96_in_Halt[:, j - 1, index])
        mse = mean_squared_error(y_test[:, j - 1, index], pred_e2d2_batch_128_96_in_Halt[:, j - 1, index])
        nse = calculate_nse(y_test[:, j - 1, index], pred_e2d2_batch_128_96_in_Halt[:, j - 1, index])
        print("MAE: ", mae)
        print("MSE: ", mse)
        print("NSE: ", nse)
    print()

value_x
time  1 :
MAE:  0.0476017583513854
MSE:  0.012030471254083649
NSE:  0.28660434887224995
time  2 :
MAE:  0.047592256709624775
MSE:  0.01297646816679875
NSE:  0.23050762005274072
time  3 :
MAE:  0.05149965125129484
MSE:  0.013683949445438798
NSE:  0.18844367225424907
time  4 :
MAE:  0.05380121383058392
MSE:  0.014274946263782073
NSE:  0.15339332297340147
time  5 :
MAE:  0.05497200518747678
MSE:  0.014771642278573563
NSE:  0.1239356875606592
time  6 :
MAE:  0.056959378622098995
MSE:  0.01520071833242816
NSE:  0.09848840072447873
time  7 :
MAE:  0.05951121502840638
MSE:  0.015493846304934333
NSE:  0.0811038099763769
time  8 :
MAE:  0.06194606661593561
MSE:  0.015704268185656713
NSE:  0.0686242835445302
time  9 :
MAE:  0.06396393291708967
MSE:  0.01592430491008623
NSE:  0.055574527934190265
time  10 :
MAE:  0.06553448531494525
MSE:  0.01616799227975689
NSE:  0.0411221194656789
time  11 :
MAE:  0.06671813326069155
MSE:  0.016355354896771597
NSE:  0.029877463114115987
time  12 :
MAE: 

now try different batch size for 48 window size input

In [46]:
n_past = 48
n_future = 48 
n_features = 2
X_train, y_train = split(train_Haltwhistle.values,n_past, n_future)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1],n_features))
y_train = y_train.reshape((y_train.shape[0], y_train.shape[1], n_features))
X_test, y_test = split(test_Haltwhistle.values,n_past, n_future)
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1],n_features))
y_test = y_test.reshape((y_test.shape[0], y_test.shape[1], n_features))
X_valid, y_valid = split(valid_Haltwhistle.values,n_past, n_future)
X_valid = X_valid.reshape((X_valid.shape[0], X_valid.shape[1],n_features))
y_valid = y_valid.reshape((y_valid.shape[0], y_valid.shape[1], n_features))

In [47]:
e_in = tf.keras.layers.Input(shape=(n_past, n_features))
e_l1 = tf.keras.layers.LSTM(100,return_sequences = True, return_state=True)
e_outs1 = e_l1(e_in)
e_states1 = e_outs1[1:]
e_l2 = tf.keras.layers.LSTM(100, return_state=True)
e_outs2 = e_l2(e_outs1[0])
e_states2 = e_outs2[1:]

d_in = tf.keras.layers.RepeatVector(n_future)(e_outs2[0])

d_l1 = tf.keras.layers.LSTM(100, return_sequences=True)(d_in,initial_state = e_states1)
d_l2 = tf.keras.layers.LSTM(100, return_sequences=True)(d_l1,initial_state = e_states2)
d_outs2 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(n_features))(d_l2)

model_e2d2_batch_32_48_in_Halt= tf.keras.models.Model(e_in,d_outs2)

In [48]:
reduce_learning_rate = tf.keras.callbacks.LearningRateScheduler(lambda x: 1e-3 * 0.90 ** x)
model_e2d2_batch_32_48_in_Halt.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.Huber())
history_e2d2_batch_32_48_in_Halt=model_e2d2_batch_32_48_in_Halt.fit(X_train,y_train,epochs=25,validation_data=(X_valid,y_valid),batch_size=32,verbose=1,callbacks=[reduce_learning_rate])

Epoch 1/25
2822/2822 [==============================] - 403s 141ms/step - loss: 0.0029 - val_loss: 0.0040 - lr: 0.0010
Epoch 2/25
2822/2822 [==============================] - 394s 140ms/step - loss: 0.0019 - val_loss: 0.0027 - lr: 9.0000e-04
Epoch 3/25
2822/2822 [==============================] - 398s 141ms/step - loss: 0.0017 - val_loss: 0.0025 - lr: 8.1000e-04
Epoch 4/25
2822/2822 [==============================] - 395s 140ms/step - loss: 0.0016 - val_loss: 0.0025 - lr: 7.2900e-04
Epoch 5/25
2822/2822 [==============================] - 394s 140ms/step - loss: 0.0016 - val_loss: 0.0024 - lr: 6.5610e-04
Epoch 6/25
2822/2822 [==============================] - 427s 151ms/step - loss: 0.0015 - val_loss: 0.0027 - lr: 5.9049e-04
Epoch 7/25
2822/2822 [==============================] - 371s 132ms/step - loss: 0.0015 - val_loss: 0.0024 - lr: 5.3144e-04
Epoch 8/25
2822/2822 [==============================] - 370s 131ms/step - loss: 0.0015 - val_loss: 0.0024 - lr: 4.7830e-04
Epoch 9/25
2822/2822

In [49]:
with open('history_e2d2_batch_32_48_in_Halt.pkl', 'wb') as file:
    pickle.dump(history_e2d2_batch_32_48_in_Halt.history, file)

In [50]:
from keras.models import load_model
model_e2d2_batch_32_48_in_Halt.save('model_e2d2_batch_32_48_in_Halt.h5')

In [51]:
pred_e2d2_batch_32_48_in_Halt=model_e2d2_batch_32_48_in_Halt.predict(X_test)

351/351 [==============================] - 16s 42ms/step


In [52]:
for index,i in enumerate(train_merged.columns):
    scaler = scalers_Haltwhistle['scaler_'+i]
    pred_e2d2_batch_32_48_in_Halt[:,:,index]=scaler.inverse_transform(pred_e2d2_batch_32_48_in_Halt[:,:,index])
    y_train[:,:,index]=scaler.inverse_transform(y_train[:,:,index])
    y_test[:,:,index]=scaler.inverse_transform(y_test[:,:,index])
    y_valid[:,:,index]=scaler.inverse_transform(y_valid[:,:,index])

In [53]:
def calculate_nse(y_true, y_pred):
    
    return 1 - np.sum((y_true - y_pred) ** 2) / np.sum((y_true - np.mean(y_true)) ** 2)

for index, i in enumerate(train_merged.columns):
    print(i)
    for j in range(1, 49):
        print("time ", j, ":")
        mae = mean_absolute_error(y_test[:, j - 1, index], pred_e2d2_batch_32_48_in_Halt[:, j - 1, index])
        mse = mean_squared_error(y_test[:, j - 1, index], pred_e2d2_batch_32_48_in_Halt[:, j - 1, index])
        nse = calculate_nse(y_test[:, j - 1, index], pred_e2d2_batch_32_48_in_Halt[:, j - 1, index])
        print("MAE: ", mae)
        print("MSE: ", mse)
        print("NSE: ", nse)
    print()

value_x
time  1 :
MAE:  0.045007012187237724
MSE:  0.011252789397191968
NSE:  0.33034125774212253
time  2 :
MAE:  0.04903396611977561
MSE:  0.012609278642012367
NSE:  0.24961595048635488
time  3 :
MAE:  0.04978445986348145
MSE:  0.01351360368664578
NSE:  0.1957991455497381
time  4 :
MAE:  0.05396187027774184
MSE:  0.014302351585346991
NSE:  0.14886039044119548
time  5 :
MAE:  0.05242262943255908
MSE:  0.014909550053718952
NSE:  0.11272572655667068
time  6 :
MAE:  0.05412887972840664
MSE:  0.015436395425096372
NSE:  0.08137291292905335
time  7 :
MAE:  0.056189924671627596
MSE:  0.01576883908952113
NSE:  0.061589035498108236
time  8 :
MAE:  0.05799342048641616
MSE:  0.015989185263606362
NSE:  0.048476131968955616
time  9 :
MAE:  0.05902472523693953
MSE:  0.016109821435988894
NSE:  0.041297017118672286
time  10 :
MAE:  0.059448199129594016
MSE:  0.016246741583364862
NSE:  0.03314883532612645
time  11 :
MAE:  0.05952689905775175
MSE:  0.01640982627726546
NSE:  0.023443589173893553
time  12

MAE:  0.09585836600602399
MSE:  0.03278398284708544
NSE:  0.6399239638799616
time  45 :
MAE:  0.0978165076035393
MSE:  0.03373598877007121
NSE:  0.6294674577829322
time  46 :
MAE:  0.09973052692921919
MSE:  0.034682017267654214
NSE:  0.6190764555952727
time  47 :
MAE:  0.10159050561221322
MSE:  0.035618839479233666
NSE:  0.6087862633826224
time  48 :
MAE:  0.10343080174261185
MSE:  0.036552924215598116
NSE:  0.5985259597021787



In [54]:
n_past = 48
n_future = 48 
n_features = 2
X_train, y_train = split(train_Haltwhistle.values,n_past, n_future)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1],n_features))
y_train = y_train.reshape((y_train.shape[0], y_train.shape[1], n_features))
X_test, y_test = split(test_Haltwhistle.values,n_past, n_future)
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1],n_features))
y_test = y_test.reshape((y_test.shape[0], y_test.shape[1], n_features))
X_valid, y_valid = split(valid_Haltwhistle.values,n_past, n_future)
X_valid = X_valid.reshape((X_valid.shape[0], X_valid.shape[1],n_features))
y_valid = y_valid.reshape((y_valid.shape[0], y_valid.shape[1], n_features))

In [55]:
e_in = tf.keras.layers.Input(shape=(n_past, n_features))
e_l1 = tf.keras.layers.LSTM(100,return_sequences = True, return_state=True)
e_outs1 = e_l1(e_in)
e_states1 = e_outs1[1:]
e_l2 = tf.keras.layers.LSTM(100, return_state=True)
e_outs2 = e_l2(e_outs1[0])
e_states2 = e_outs2[1:]

d_in = tf.keras.layers.RepeatVector(n_future)(e_outs2[0])

d_l1 = tf.keras.layers.LSTM(100, return_sequences=True)(d_in,initial_state = e_states1)
d_l2 = tf.keras.layers.LSTM(100, return_sequences=True)(d_l1,initial_state = e_states2)
d_outs2 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(n_features))(d_l2)

model_e2d2_batch_64_48_in_Halt= tf.keras.models.Model(e_in,d_outs2)

In [56]:
reduce_learning_rate = tf.keras.callbacks.LearningRateScheduler(lambda x: 1e-3 * 0.90 ** x)
model_e2d2_batch_64_48_in_Halt.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.Huber())
history_e2d2_batch_64_48_in_Halt=model_e2d2_batch_64_48_in_Halt.fit(X_train,y_train,epochs=25,validation_data=(X_valid,y_valid),batch_size=64,verbose=1,callbacks=[reduce_learning_rate])

Epoch 1/25
1411/1411 [==============================] - 297s 206ms/step - loss: 0.0036 - val_loss: 0.0035 - lr: 0.0010
Epoch 2/25
1411/1411 [==============================] - 300s 213ms/step - loss: 0.0020 - val_loss: 0.0030 - lr: 9.0000e-04
Epoch 3/25
1411/1411 [==============================] - 298s 211ms/step - loss: 0.0018 - val_loss: 0.0028 - lr: 8.1000e-04
Epoch 4/25
1411/1411 [==============================] - 297s 211ms/step - loss: 0.0017 - val_loss: 0.0028 - lr: 7.2900e-04
Epoch 5/25
1411/1411 [==============================] - 300s 213ms/step - loss: 0.0017 - val_loss: 0.0026 - lr: 6.5610e-04
Epoch 6/25
1411/1411 [==============================] - 297s 211ms/step - loss: 0.0016 - val_loss: 0.0025 - lr: 5.9049e-04
Epoch 7/25
1411/1411 [==============================] - 297s 211ms/step - loss: 0.0016 - val_loss: 0.0025 - lr: 5.3144e-04
Epoch 8/25
1411/1411 [==============================] - 304s 215ms/step - loss: 0.0015 - val_loss: 0.0025 - lr: 4.7830e-04
Epoch 9/25
1411/1411

In [57]:
with open('history_e2d2_batch_64_48_in_Halt.pkl', 'wb') as file:
    pickle.dump(history_e2d2_batch_64_48_in_Halt.history, file)

In [58]:
from keras.models import load_model
model_e2d2_batch_64_48_in_Halt.save('model_e2d2_batch_64_48_in_Halt.h5')

In [59]:
pred_e2d2_batch_64_48_in_Halt=model_e2d2_batch_64_48_in_Halt.predict(X_test)

351/351 [==============================] - 19s 50ms/step


In [60]:
for index,i in enumerate(train_merged.columns):
    scaler = scalers_Haltwhistle['scaler_'+i]
    pred_e2d2_batch_64_48_in_Halt[:,:,index]=scaler.inverse_transform(pred_e2d2_batch_64_48_in_Halt[:,:,index])
    y_train[:,:,index]=scaler.inverse_transform(y_train[:,:,index])
    y_test[:,:,index]=scaler.inverse_transform(y_test[:,:,index])
    y_valid[:,:,index]=scaler.inverse_transform(y_valid[:,:,index])

In [61]:
def calculate_nse(y_true, y_pred):
    
    return 1 - np.sum((y_true - y_pred) ** 2) / np.sum((y_true - np.mean(y_true)) ** 2)

for index, i in enumerate(train_merged.columns):
    print(i)
    for j in range(1, 49):
        print("time ", j, ":")
        mae = mean_absolute_error(y_test[:, j - 1, index], pred_e2d2_batch_64_48_in_Halt[:, j - 1, index])
        mse = mean_squared_error(y_test[:, j - 1, index], pred_e2d2_batch_64_48_in_Halt[:, j - 1, index])
        nse = calculate_nse(y_test[:, j - 1, index], pred_e2d2_batch_64_48_in_Halt[:, j - 1, index])
        print("MAE: ", mae)
        print("MSE: ", mse)
        print("NSE: ", nse)
    print()

value_x
time  1 :
MAE:  0.04487469189297882
MSE:  0.011582436093197255
NSE:  0.3107238292055633
time  2 :
MAE:  0.04624350277987084
MSE:  0.013004587829878152
NSE:  0.22609091645210522
time  3 :
MAE:  0.04680690654310358
MSE:  0.013720277193665654
NSE:  0.18349990881084643
time  4 :
MAE:  0.05049059373361094
MSE:  0.014273763602685419
NSE:  0.15056167461502135
time  5 :
MAE:  0.052363535269516384
MSE:  0.014750080941100703
NSE:  0.12221580778147034
time  6 :
MAE:  0.05455763037637748
MSE:  0.015197726476013425
NSE:  0.09557621333906363
time  7 :
MAE:  0.056511871949842286
MSE:  0.015503009140525626
NSE:  0.07740869967339492
time  8 :
MAE:  0.05791395518036093
MSE:  0.01571804555481053
NSE:  0.06461178242498333
time  9 :
MAE:  0.05895430536793579
MSE:  0.0160003652623887
NSE:  0.047810805030117476
time  10 :
MAE:  0.05958614838756339
MSE:  0.016344637323728792
NSE:  0.027323014185203665
time  11 :
MAE:  0.060017380605899354
MSE:  0.01659807452117227
NSE:  0.012240848431376183
time  12 :

MAE:  0.09164593517170815
MSE:  0.032932543614310014
NSE:  0.6382922776862894
time  45 :
MAE:  0.09361723462940809
MSE:  0.03384397960764077
NSE:  0.6282813618349665
time  46 :
MAE:  0.09560766011878373
MSE:  0.0347501415082481
NSE:  0.618328225554712
time  47 :
MAE:  0.09757359233185775
MSE:  0.03564718236096932
NSE:  0.6084749639457956
time  48 :
MAE:  0.09950957938765785
MSE:  0.0365396078871039
NSE:  0.5986722177736525



In [62]:
n_past = 48
n_future = 48 
n_features = 2
X_train, y_train = split(train_Haltwhistle.values,n_past, n_future)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1],n_features))
y_train = y_train.reshape((y_train.shape[0], y_train.shape[1], n_features))
X_test, y_test = split(test_Haltwhistle.values,n_past, n_future)
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1],n_features))
y_test = y_test.reshape((y_test.shape[0], y_test.shape[1], n_features))
X_valid, y_valid = split(valid_Haltwhistle.values,n_past, n_future)
X_valid = X_valid.reshape((X_valid.shape[0], X_valid.shape[1],n_features))
y_valid = y_valid.reshape((y_valid.shape[0], y_valid.shape[1], n_features))

In [63]:
e_in = tf.keras.layers.Input(shape=(n_past, n_features))
e_l1 = tf.keras.layers.LSTM(100,return_sequences = True, return_state=True)
e_outs1 = e_l1(e_in)
e_states1 = e_outs1[1:]
e_l2 = tf.keras.layers.LSTM(100, return_state=True)
e_outs2 = e_l2(e_outs1[0])
e_states2 = e_outs2[1:]

d_in = tf.keras.layers.RepeatVector(n_future)(e_outs2[0])

d_l1 = tf.keras.layers.LSTM(100, return_sequences=True)(d_in,initial_state = e_states1)
d_l2 = tf.keras.layers.LSTM(100, return_sequences=True)(d_l1,initial_state = e_states2)
d_outs2 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(n_features))(d_l2)

model_e2d2_batch_128_48_in_Halt= tf.keras.models.Model(e_in,d_outs2)

In [64]:
reduce_learning_rate = tf.keras.callbacks.LearningRateScheduler(lambda x: 1e-3 * 0.90 ** x)
model_e2d2_batch_128_48_in_Halt.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.Huber())
history_e2d2_batch_128_48_in_Halt=model_e2d2_batch_128_48_in_Halt.fit(X_train,y_train,epochs=25,validation_data=(X_valid,y_valid),batch_size=128,verbose=1,callbacks=[reduce_learning_rate])

Epoch 1/25
706/706 [==============================] - 282s 391ms/step - loss: 0.0051 - val_loss: 0.0037 - lr: 0.0010
Epoch 2/25
706/706 [==============================] - 273s 387ms/step - loss: 0.0021 - val_loss: 0.0029 - lr: 9.0000e-04
Epoch 3/25
706/706 [==============================] - 281s 399ms/step - loss: 0.0019 - val_loss: 0.0028 - lr: 8.1000e-04
Epoch 4/25
706/706 [==============================] - 279s 396ms/step - loss: 0.0018 - val_loss: 0.0026 - lr: 7.2900e-04
Epoch 5/25
706/706 [==============================] - 278s 394ms/step - loss: 0.0017 - val_loss: 0.0025 - lr: 6.5610e-04
Epoch 6/25
706/706 [==============================] - 278s 393ms/step - loss: 0.0016 - val_loss: 0.0026 - lr: 5.9049e-04
Epoch 7/25
706/706 [==============================] - 278s 394ms/step - loss: 0.0016 - val_loss: 0.0024 - lr: 5.3144e-04
Epoch 8/25
706/706 [==============================] - 279s 395ms/step - loss: 0.0016 - val_loss: 0.0024 - lr: 4.7830e-04
Epoch 9/25
706/706 [================

In [65]:
with open('history_e2d2_batch_128_48_in_Halt.pkl', 'wb') as file:
    pickle.dump(history_e2d2_batch_128_48_in_Halt.history, file)

In [66]:
from keras.models import load_model
model_e2d2_batch_128_48_in_Halt.save('model_e2d2_batch_128_48_in_Halt.h5')

In [67]:
pred_e2d2_batch_128_48_in_Halt=model_e2d2_batch_128_48_in_Halt.predict(X_test)

351/351 [==============================] - 19s 51ms/step


In [68]:
for index,i in enumerate(train_merged.columns):
    scaler = scalers_Haltwhistle['scaler_'+i]
    pred_e2d2_batch_128_48_in_Halt[:,:,index]=scaler.inverse_transform(pred_e2d2_batch_128_48_in_Halt[:,:,index])
    y_train[:,:,index]=scaler.inverse_transform(y_train[:,:,index])
    y_test[:,:,index]=scaler.inverse_transform(y_test[:,:,index])
    y_valid[:,:,index]=scaler.inverse_transform(y_valid[:,:,index])

In [69]:
def calculate_nse(y_true, y_pred):
    
    return 1 - np.sum((y_true - y_pred) ** 2) / np.sum((y_true - np.mean(y_true)) ** 2)

for index, i in enumerate(train_merged.columns):
    print(i)
    for j in range(1, 49):
        print("time ", j, ":")
        mae = mean_absolute_error(y_test[:, j - 1, index], pred_e2d2_batch_128_48_in_Halt[:, j - 1, index])
        mse = mean_squared_error(y_test[:, j - 1, index], pred_e2d2_batch_128_48_in_Halt[:, j - 1, index])
        nse = calculate_nse(y_test[:, j - 1, index], pred_e2d2_batch_128_48_in_Halt[:, j - 1, index])
        print("MAE: ", mae)
        print("MSE: ", mse)
        print("NSE: ", nse)
    print()

value_x
time  1 :
MAE:  0.047715968694180474
MSE:  0.011632897775647516
NSE:  0.30772083096138647
time  2 :
MAE:  0.05427851552252577
MSE:  0.012918507084665005
NSE:  0.2312136217243146
time  3 :
MAE:  0.05569596882964395
MSE:  0.01377519340523691
NSE:  0.18023181946230227
time  4 :
MAE:  0.0589424609882605
MSE:  0.014511138506038183
NSE:  0.13643538346960415
time  5 :
MAE:  0.061193442206359354
MSE:  0.015071389287987232
NSE:  0.1030945982876994
time  6 :
MAE:  0.06241112891609571
MSE:  0.015500400916984859
NSE:  0.07756391627207004
time  7 :
MAE:  0.06302277942897487
MSE:  0.015750591829438724
NSE:  0.06267493845115613
time  8 :
MAE:  0.06333283988241106
MSE:  0.015897324126264545
NSE:  0.05394283107114006
time  9 :
MAE:  0.06361934566540359
MSE:  0.016057644320803935
NSE:  0.04440210156450086
time  10 :
MAE:  0.06395757506650136
MSE:  0.016279855710233986
NSE:  0.031178198200750407
time  11 :
MAE:  0.06432969672932044
MSE:  0.01646830792750654
NSE:  0.019963318908160277
time  12 :
M

now try different batch size for 24 inputs

In [70]:
n_past = 24
n_future = 48 
n_features = 2
X_train, y_train = split(train_Haltwhistle.values,n_past, n_future)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1],n_features))
y_train = y_train.reshape((y_train.shape[0], y_train.shape[1], n_features))
X_test, y_test = split(test_Haltwhistle.values,n_past, n_future)
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1],n_features))
y_test = y_test.reshape((y_test.shape[0], y_test.shape[1], n_features))
X_valid, y_valid = split(valid_Haltwhistle.values,n_past, n_future)
X_valid = X_valid.reshape((X_valid.shape[0], X_valid.shape[1],n_features))
y_valid = y_valid.reshape((y_valid.shape[0], y_valid.shape[1], n_features))

In [71]:
e_in = tf.keras.layers.Input(shape=(n_past, n_features))
e_l1 = tf.keras.layers.LSTM(100,return_sequences = True, return_state=True)
e_outs1 = e_l1(e_in)
e_states1 = e_outs1[1:]
e_l2 = tf.keras.layers.LSTM(100, return_state=True)
e_outs2 = e_l2(e_outs1[0])
e_states2 = e_outs2[1:]

d_in = tf.keras.layers.RepeatVector(n_future)(e_outs2[0])

d_l1 = tf.keras.layers.LSTM(100, return_sequences=True)(d_in,initial_state = e_states1)
d_l2 = tf.keras.layers.LSTM(100, return_sequences=True)(d_l1,initial_state = e_states2)
d_outs2 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(n_features))(d_l2)

model_e2d2_batch_32_24_in_Halt= tf.keras.models.Model(e_in,d_outs2)

In [72]:
reduce_learning_rate = tf.keras.callbacks.LearningRateScheduler(lambda x: 1e-3 * 0.90 ** x)
model_e2d2_batch_32_24_in_Halt.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.Huber())
history_e2d2_batch_32_24_in_Halt=model_e2d2_batch_32_24_in_Halt.fit(X_train,y_train,epochs=25,validation_data=(X_valid,y_valid),batch_size=32,verbose=1,callbacks=[reduce_learning_rate])

Epoch 1/25
2823/2823 [==============================] - 367s 128ms/step - loss: 0.0030 - val_loss: 0.0030 - lr: 0.0010
Epoch 2/25
2823/2823 [==============================] - 350s 124ms/step - loss: 0.0019 - val_loss: 0.0027 - lr: 9.0000e-04
Epoch 3/25
2823/2823 [==============================] - 350s 124ms/step - loss: 0.0018 - val_loss: 0.0027 - lr: 8.1000e-04
Epoch 4/25
2823/2823 [==============================] - 352s 125ms/step - loss: 0.0017 - val_loss: 0.0026 - lr: 7.2900e-04
Epoch 5/25
2823/2823 [==============================] - 349s 124ms/step - loss: 0.0016 - val_loss: 0.0026 - lr: 6.5610e-04
Epoch 6/25
2823/2823 [==============================] - 350s 124ms/step - loss: 0.0016 - val_loss: 0.0024 - lr: 5.9049e-04
Epoch 7/25
2823/2823 [==============================] - 350s 124ms/step - loss: 0.0015 - val_loss: 0.0025 - lr: 5.3144e-04
Epoch 8/25
2823/2823 [==============================] - 348s 123ms/step - loss: 0.0015 - val_loss: 0.0025 - lr: 4.7830e-04
Epoch 9/25
2823/2823

In [73]:
with open('history_e2d2_batch_32_24_in_Halt.pkl', 'wb') as file:
    pickle.dump(history_e2d2_batch_32_24_in_Halt.history, file)

In [74]:
from keras.models import load_model
model_e2d2_batch_32_24_in_Halt.save('model_e2d2_batch_32_24_in_Halt.h5')

In [75]:
pred_e2d2_batch_32_24_in_Halt=model_e2d2_batch_32_24_in_Halt.predict(X_test)

351/351 [==============================] - 15s 40ms/step


In [76]:
for index,i in enumerate(train_merged.columns):
    scaler = scalers_Haltwhistle['scaler_'+i]
    pred_e2d2_batch_32_24_in_Halt[:,:,index]=scaler.inverse_transform(pred_e2d2_batch_32_24_in_Halt[:,:,index])
    y_train[:,:,index]=scaler.inverse_transform(y_train[:,:,index])
    y_test[:,:,index]=scaler.inverse_transform(y_test[:,:,index])
    y_valid[:,:,index]=scaler.inverse_transform(y_valid[:,:,index])

In [77]:
def calculate_nse(y_true, y_pred):
    
    return 1 - np.sum((y_true - y_pred) ** 2) / np.sum((y_true - np.mean(y_true)) ** 2)

for index, i in enumerate(train_merged.columns):
    print(i)
    for j in range(1, 49):
        print("time ", j, ":")
        mae = mean_absolute_error(y_test[:, j - 1, index], pred_e2d2_batch_32_24_in_Halt[:, j - 1, index])
        mse = mean_squared_error(y_test[:, j - 1, index], pred_e2d2_batch_32_24_in_Halt[:, j - 1, index])
        nse = calculate_nse(y_test[:, j - 1, index], pred_e2d2_batch_32_24_in_Halt[:, j - 1, index])
        print("MAE: ", mae)
        print("MSE: ", mse)
        print("NSE: ", nse)
    print()

value_x
time  1 :
MAE:  0.05278172002937249
MSE:  0.011086764536573266
NSE:  0.33891480854646605
time  2 :
MAE:  0.04840043280510883
MSE:  0.012560644704910431
NSE:  0.2510298038590749
time  3 :
MAE:  0.049776027544235456
MSE:  0.013312952801294877
NSE:  0.20617093269880815
time  4 :
MAE:  0.05354006598517863
MSE:  0.014054568666906206
NSE:  0.16194962134279212
time  5 :
MAE:  0.05676576422184232
MSE:  0.014701639916604415
NSE:  0.12336584700721531
time  6 :
MAE:  0.05924557976650952
MSE:  0.015215571742163782
NSE:  0.09272095343405495
time  7 :
MAE:  0.060637646730399036
MSE:  0.015638919743229153
NSE:  0.06747742152602776
time  8 :
MAE:  0.061306476830820425
MSE:  0.015935611546013095
NSE:  0.049786186486346695
time  9 :
MAE:  0.06168791591054198
MSE:  0.01620050410787865
NSE:  0.033991086897297906
time  10 :
MAE:  0.061989752723353456
MSE:  0.01641482857866528
NSE:  0.02121127784337984
time  11 :
MAE:  0.06231781208076719
MSE:  0.016576845923797358
NSE:  0.011550455042272922
time  1

MAE:  0.0949841100795018
MSE:  0.03380509455489154
NSE:  0.6279670763855294
time  46 :
MAE:  0.09697475853735579
MSE:  0.034717195289376106
NSE:  0.6179306646334727
time  47 :
MAE:  0.09898237467104665
MSE:  0.035620479892943584
NSE:  0.6079912469435329
time  48 :
MAE:  0.10096129707677606
MSE:  0.036521157247824544
NSE:  0.598080385704294



In [78]:
n_past = 24
n_future = 48 
n_features = 2
X_train, y_train = split(train_Haltwhistle.values,n_past, n_future)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1],n_features))
y_train = y_train.reshape((y_train.shape[0], y_train.shape[1], n_features))
X_test, y_test = split(test_Haltwhistle.values,n_past, n_future)
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1],n_features))
y_test = y_test.reshape((y_test.shape[0], y_test.shape[1], n_features))
X_valid, y_valid = split(valid_Haltwhistle.values,n_past, n_future)
X_valid = X_valid.reshape((X_valid.shape[0], X_valid.shape[1],n_features))
y_valid = y_valid.reshape((y_valid.shape[0], y_valid.shape[1], n_features))

In [79]:
e_in = tf.keras.layers.Input(shape=(n_past, n_features))
e_l1 = tf.keras.layers.LSTM(100,return_sequences = True, return_state=True)
e_outs1 = e_l1(e_in)
e_states1 = e_outs1[1:]
e_l2 = tf.keras.layers.LSTM(100, return_state=True)
e_outs2 = e_l2(e_outs1[0])
e_states2 = e_outs2[1:]

d_in = tf.keras.layers.RepeatVector(n_future)(e_outs2[0])

d_l1 = tf.keras.layers.LSTM(100, return_sequences=True)(d_in,initial_state = e_states1)
d_l2 = tf.keras.layers.LSTM(100, return_sequences=True)(d_l1,initial_state = e_states2)
d_outs2 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(n_features))(d_l2)

model_e2d2_batch_64_24_in_Halt= tf.keras.models.Model(e_in,d_outs2)

In [80]:
reduce_learning_rate = tf.keras.callbacks.LearningRateScheduler(lambda x: 1e-3 * 0.90 ** x)
model_e2d2_batch_64_24_in_Halt.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.Huber())
history_e2d2_batch_64_24_in_Halt=model_e2d2_batch_64_24_in_Halt.fit(X_train,y_train,epochs=25,validation_data=(X_valid,y_valid),batch_size=64,verbose=1,callbacks=[reduce_learning_rate])

Epoch 1/25
1412/1412 [==============================] - 281s 195ms/step - loss: 0.0035 - val_loss: 0.0033 - lr: 0.0010
Epoch 2/25
1412/1412 [==============================] - 316s 224ms/step - loss: 0.0020 - val_loss: 0.0027 - lr: 9.0000e-04
Epoch 3/25
1412/1412 [==============================] - 315s 223ms/step - loss: 0.0018 - val_loss: 0.0026 - lr: 8.1000e-04
Epoch 4/25
1412/1412 [==============================] - 314s 222ms/step - loss: 0.0018 - val_loss: 0.0025 - lr: 7.2900e-04
Epoch 5/25
1412/1412 [==============================] - 316s 224ms/step - loss: 0.0017 - val_loss: 0.0031 - lr: 6.5610e-04
Epoch 6/25
1412/1412 [==============================] - 315s 223ms/step - loss: 0.0016 - val_loss: 0.0026 - lr: 5.9049e-04
Epoch 7/25
1412/1412 [==============================] - 319s 226ms/step - loss: 0.0016 - val_loss: 0.0025 - lr: 5.3144e-04
Epoch 8/25
1412/1412 [==============================] - 317s 224ms/step - loss: 0.0016 - val_loss: 0.0024 - lr: 4.7830e-04
Epoch 9/25
1412/1412

In [81]:
with open('history_e2d2_batch_64_24_in_Halt.pkl', 'wb') as file:
    pickle.dump(history_e2d2_batch_64_24_in_Halt.history, file)

In [82]:
from keras.models import load_model
model_e2d2_batch_64_24_in_Halt.save('model_e2d2_batch_64_24_in_Halt.h5')

In [83]:
pred_e2d2_batch_64_24_in_Halt=model_e2d2_batch_64_24_in_Halt.predict(X_test)

351/351 [==============================] - 23s 58ms/step


In [84]:
for index,i in enumerate(train_merged.columns):
    scaler = scalers_Haltwhistle['scaler_'+i]
    pred_e2d2_batch_64_24_in_Halt[:,:,index]=scaler.inverse_transform(pred_e2d2_batch_64_24_in_Halt[:,:,index])
    y_train[:,:,index]=scaler.inverse_transform(y_train[:,:,index])
    y_test[:,:,index]=scaler.inverse_transform(y_test[:,:,index])
    y_valid[:,:,index]=scaler.inverse_transform(y_valid[:,:,index])

In [85]:
def calculate_nse(y_true, y_pred):
    
    return 1 - np.sum((y_true - y_pred) ** 2) / np.sum((y_true - np.mean(y_true)) ** 2)

for index, i in enumerate(train_merged.columns):
    print(i)
    for j in range(1, 49):
        print("time ", j, ":")
        mae = mean_absolute_error(y_test[:, j - 1, index], pred_e2d2_batch_64_24_in_Halt[:, j - 1, index])
        mse = mean_squared_error(y_test[:, j - 1, index], pred_e2d2_batch_64_24_in_Halt[:, j - 1, index])
        nse = calculate_nse(y_test[:, j - 1, index], pred_e2d2_batch_64_24_in_Halt[:, j - 1, index])
        print("MAE: ", mae)
        print("MSE: ", mse)
        print("NSE: ", nse)
    print()

value_x
time  1 :
MAE:  0.04442674411097982
MSE:  0.011123377001645741
NSE:  0.33673166860494574
time  2 :
MAE:  0.04392086647849695
MSE:  0.012438599757164267
NSE:  0.25830713958498075
time  3 :
MAE:  0.04056915759997819
MSE:  0.013417019627953598
NSE:  0.1999656022077806
time  4 :
MAE:  0.04320649906219257
MSE:  0.01409087597425338
NSE:  0.15978467744508507
time  5 :
MAE:  0.044082581462922966
MSE:  0.014680670242306567
NSE:  0.1246162335471459
time  6 :
MAE:  0.04485607142970319
MSE:  0.015219726095377922
NSE:  0.09247323631325965
time  7 :
MAE:  0.04531872538958031
MSE:  0.015579900448226073
NSE:  0.07099664318964782
time  8 :
MAE:  0.045610244162244684
MSE:  0.015795691978264092
NSE:  0.058129355850890074
time  9 :
MAE:  0.04597321509550577
MSE:  0.016012635763368404
NSE:  0.04519336147333963
time  10 :
MAE:  0.046452106377651486
MSE:  0.01628526410924826
NSE:  0.028936990046211752
time  11 :
MAE:  0.04703177367072888
MSE:  0.016460972071802558
NSE:  0.018459818669302464
time  12 

In [86]:
n_past = 24
n_future = 48 
n_features = 2
X_train, y_train = split(train_Haltwhistle.values,n_past, n_future)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1],n_features))
y_train = y_train.reshape((y_train.shape[0], y_train.shape[1], n_features))
X_test, y_test = split(test_Haltwhistle.values,n_past, n_future)
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1],n_features))
y_test = y_test.reshape((y_test.shape[0], y_test.shape[1], n_features))
X_valid, y_valid = split(valid_Haltwhistle.values,n_past, n_future)
X_valid = X_valid.reshape((X_valid.shape[0], X_valid.shape[1],n_features))
y_valid = y_valid.reshape((y_valid.shape[0], y_valid.shape[1], n_features))

In [87]:
e_in = tf.keras.layers.Input(shape=(n_past, n_features))
e_l1 = tf.keras.layers.LSTM(100,return_sequences = True, return_state=True)
e_outs1 = e_l1(e_in)
e_states1 = e_outs1[1:]
e_l2 = tf.keras.layers.LSTM(100, return_state=True)
e_outs2 = e_l2(e_outs1[0])
e_states2 = e_outs2[1:]

d_in = tf.keras.layers.RepeatVector(n_future)(e_outs2[0])

d_l1 = tf.keras.layers.LSTM(100, return_sequences=True)(d_in,initial_state = e_states1)
d_l2 = tf.keras.layers.LSTM(100, return_sequences=True)(d_l1,initial_state = e_states2)
d_outs2 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(n_features))(d_l2)

model_e2d2_batch_128_24_in_Halt= tf.keras.models.Model(e_in,d_outs2)

In [88]:
reduce_learning_rate = tf.keras.callbacks.LearningRateScheduler(lambda x: 1e-3 * 0.90 ** x)
model_e2d2_batch_128_24_in_Halt.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.Huber())
history_e2d2_batch_128_24_in_Halt=model_e2d2_batch_128_24_in_Halt.fit(X_train,y_train,epochs=25,validation_data=(X_valid,y_valid),batch_size=128,verbose=1,callbacks=[reduce_learning_rate])

Epoch 1/25
706/706 [==============================] - 287s 393ms/step - loss: 0.0046 - val_loss: 0.0035 - lr: 0.0010
Epoch 2/25
706/706 [==============================] - 275s 390ms/step - loss: 0.0021 - val_loss: 0.0032 - lr: 9.0000e-04
Epoch 3/25
706/706 [==============================] - 286s 406ms/step - loss: 0.0020 - val_loss: 0.0028 - lr: 8.1000e-04
Epoch 4/25
706/706 [==============================] - 283s 401ms/step - loss: 0.0019 - val_loss: 0.0027 - lr: 7.2900e-04
Epoch 5/25
706/706 [==============================] - 269s 381ms/step - loss: 0.0018 - val_loss: 0.0030 - lr: 6.5610e-04
Epoch 6/25
706/706 [==============================] - 244s 345ms/step - loss: 0.0017 - val_loss: 0.0025 - lr: 5.9049e-04
Epoch 7/25
706/706 [==============================] - 244s 346ms/step - loss: 0.0017 - val_loss: 0.0026 - lr: 5.3144e-04
Epoch 8/25
706/706 [==============================] - 246s 349ms/step - loss: 0.0016 - val_loss: 0.0025 - lr: 4.7830e-04
Epoch 9/25
706/706 [================

In [89]:
with open('history_e2d2_batch_128_24_in_Halt.pkl', 'wb') as file:
    pickle.dump(history_e2d2_batch_128_24_in_Halt.history, file)

In [90]:
from keras.models import load_model
model_e2d2_batch_128_24_in_Halt.save('model_e2d2_batch_128_24_in_Halt.h5')

In [91]:
pred_e2d2_batch_128_24_in_Halt=model_e2d2_batch_128_24_in_Halt.predict(X_test)

351/351 [==============================] - 21s 46ms/step


In [92]:
for index,i in enumerate(train_merged.columns):
    scaler = scalers_Haltwhistle['scaler_'+i]
    pred_e2d2_batch_128_24_in_Halt[:,:,index]=scaler.inverse_transform(pred_e2d2_batch_128_24_in_Halt[:,:,index])
    y_train[:,:,index]=scaler.inverse_transform(y_train[:,:,index])
    y_test[:,:,index]=scaler.inverse_transform(y_test[:,:,index])
    y_valid[:,:,index]=scaler.inverse_transform(y_valid[:,:,index])

In [93]:
def calculate_nse(y_true, y_pred):
    
    return 1 - np.sum((y_true - y_pred) ** 2) / np.sum((y_true - np.mean(y_true)) ** 2)

for index, i in enumerate(train_merged.columns):
    print(i)
    for j in range(1, 49):
        print("time ", j, ":")
        mae = mean_absolute_error(y_test[:, j - 1, index], pred_e2d2_batch_128_24_in_Halt[:, j - 1, index])
        mse = mean_squared_error(y_test[:, j - 1, index], pred_e2d2_batch_128_24_in_Halt[:, j - 1, index])
        nse = calculate_nse(y_test[:, j - 1, index], pred_e2d2_batch_128_24_in_Halt[:, j - 1, index])
        print("MAE: ", mae)
        print("MSE: ", mse)
        print("NSE: ", nse)
    print()

value_x
time  1 :
MAE:  0.054548939958749564
MSE:  0.011611133041937685
NSE:  0.30764759324505897
time  2 :
MAE:  0.05542580410234332
MSE:  0.012708644002786207
NSE:  0.242204853726109
time  3 :
MAE:  0.05404290388163185
MSE:  0.013423281389070923
NSE:  0.19959222388505815
time  4 :
MAE:  0.05536552850959542
MSE:  0.014037767061700497
NSE:  0.16295147290710577
time  5 :
MAE:  0.05687682776196747
MSE:  0.014545079017511427
NSE:  0.13270131107426553
time  6 :
MAE:  0.05797163633960939
MSE:  0.014971362878059056
NSE:  0.10728271878487039
time  7 :
MAE:  0.05847176084196551
MSE:  0.015248816721072166
NSE:  0.09073861104967251
time  8 :
MAE:  0.05849579880228876
MSE:  0.01542862600487953
NSE:  0.08001688475894209
time  9 :
MAE:  0.058288086039245644
MSE:  0.015608168469849141
NSE:  0.06931106842840418
time  10 :
MAE:  0.05806368115175673
MSE:  0.015817861659714084
NSE:  0.05680741489530439
time  11 :
MAE:  0.05787187574255636
MSE:  0.01597621471289802
NSE:  0.04736508768288661
time  12 :
MA

batch 128 with 24 window size gives the best overall result